test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from PETsARD import (
    Loader,
    Splitter,
    Processor,
    Synthesizer,
    Evaluator,
    Executor
)

In [4]:
load = Loader(
    filepath='benchmark://adult-income'
)
load.load()

Loader - Benchmarker: file benchmark/adult-income.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [5]:
sp = Splitter()
sp.split(load.data)

In [6]:
synth = Synthesizer(method='sdv-gaussiancopula')

In [7]:
synth.create(sp.data[1]['train'], metadata=load.metadata.to_sdv())

In [8]:
synth.fit_sample()

Synthesizer (SDV): Fitting GaussianCopula.
Synthesizer (SDV): Fitting GaussianCopula spent 6.0296 sec.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:700: RuntimeWarning: invalid value encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


Synthesizer (SDV): Sampling GaussianCopula # 39073 rows (same as raw) in 0.6165 sec.


In [9]:
synth.data_syn

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,42,Federal-gov,130228,Some-college,11,Divorced,Craft-repair,Other-relative,White,Male,13195,1,39,United-States,<=50K
1,49,?,176802,Bachelors,10,Never-married,Exec-managerial,Own-child,White,Male,19305,8,13,United-States,<=50K
2,30,Private,40180,Some-college,11,Never-married,Other-service,Own-child,White,Female,2105,0,36,Holand-Netherlands,<=50K
3,23,Private,191441,Some-college,14,Married-civ-spouse,Craft-repair,Not-in-family,White,Male,75399,0,50,United-States,<=50K
4,45,Private,159730,Some-college,7,Married-AF-spouse,Sales,Wife,White,Female,10226,0,37,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,23,Local-gov,242668,Bachelors,12,Never-married,Prof-specialty,Own-child,White,Male,12363,0,50,United-States,<=50K
39069,30,Private,370013,Bachelors,10,Never-married,Sales,Unmarried,White,Male,4671,419,38,United-States,<=50K
39070,54,Private,163519,Bachelors,7,Married-spouse-absent,Exec-managerial,Unmarried,White,Female,13061,0,47,Holand-Netherlands,<=50K
39071,52,Private,126712,HS-grad,9,Divorced,Prof-specialty,Husband,White,Male,37315,0,17,United-States,<=50K


In [10]:
ml = Evaluator('mlutility-classification', target='income')

In [11]:
ml.create({'syn': synth.data_syn, 'ori': sp.data[1]['train'], 'control': sp.data[1]['validation']})

In [12]:
ml.eval()

In [13]:
ml.get_global()

,ori_mean,ori_std,syn_mean,syn_std,pct_change
0,0.856894,0.006159,0.765662,0.00419,-9.1232
